In [1]:
using DynamicPolynomials
using TSLTSSOS
using MAT
using JuMP
include("./Control_utils.jl")

Main.Control

In [2]:
function CartPoleTO(cartpole_param)
    ## Double inverted pendulum # nearly optimal!
    Ns = cartpole_param.control_param.Ns  # 80
    dof = 2

    lx = cartpole_param.lx
    ly = cartpole_param.ly

    dt = cartpole_param.control_param.dt
    m = cartpole_param.mass
    I = cartpole_param.inertial
    g = -9.81

    xc_0 = cartpole_param.xc_0
    yc_0 = 0.0

    t0 = cartpole_param.theta_0

    @polyvar x[1:dof+1, 1:Ns+1]
    @polyvar y[1:dof, 1:Ns+1]
    @polyvar c[1:dof, 1:Ns+1]
    @polyvar s[1:dof, 1:Ns+1]

    @polyvar vx[1:dof+1, 1:Ns+1]
    @polyvar vy[1:dof, 1:Ns+1]
    @polyvar ca[1:dof, 1:Ns+1]
    @polyvar sa[1:dof, 1:Ns+1]

    @polyvar lam_x[1:dof, 1:Ns] # serial manipulator
    @polyvar lam_y[1:dof, 1:Ns]
    @polyvar tau[1:Ns]


    ns = dof * (Ns+1)
    nf = Ns
    nlam = dof * Ns
    var = append!( reshape(x, ns + Ns + 1),  reshape(y, ns),  reshape(c, ns),  reshape(s, ns),
                   reshape(vx, ns + Ns + 1), reshape(vy, ns), reshape(ca, ns), reshape(sa, ns),
                   reshape(lam_x, nlam), reshape(lam_y, nlam), reshape(tau, nf))

    eq = []
    ineq = []

    f = 0

    # cart states
    append!( eq, [ x[end, 1] - xc_0])
    append!( eq, [vx[end, 1] - 0.0])

    # pole states
    for k in 1:dof
        append!(eq, [c[k, 1] - cos(t0[k])])
        append!(eq, [s[k, 1] - sin(t0[k])])
        append!(eq, [vx[k, 1] - 0.0])
        append!(eq, [vy[k, 1] - 0.0])
        append!(eq, [ca[k, 1] - 1.0])
        append!(eq, [sa[k, 1] - 0.0])

        if k == 1
            append!(eq, [cos(t0[k]) * lx - sin(t0[k]) * ly + x[k, 1] - x[end, 1] ])
            append!(eq, [sin(t0[k]) * lx + cos(t0[k]) * ly + y[k, 1] - 0 ])
        else
            append!(eq, [cos(t0[k]) * lx - sin(t0[k]) * ly + x[k, 1] - x[k-1, 1]])
            append!(eq, [sin(t0[k]) * lx + cos(t0[k]) * ly + y[k, 1] - y[k-1, 1]])
        end
    end

    for k in 1:Ns
        
        ######## Cart Start ########
        append!( eq, [x[end, k+1] - (x[end, k] + dt * vx[end, k])] )
        append!( eq, [m * vx[end, k+1] - m * vx[end, k] - (lam_x[1, k] + tau[k])* dt ] )
        ######## Cart End ########

        for j in 1:dof

            ######## kinematics ########
            append!( eq, [c[j, k+1] -       (c[j, k] * ca[j, k] - s[j, k] * sa[j, k])] )
            append!( eq, [s[j, k+1] -       (s[j, k] * ca[j, k] + c[j, k] * sa[j, k])] )
            append!( eq, [x[j, k+1] - (dt * (c[j, k] * vx[j, k] - s[j, k] * vy[j, k]) + x[j, k])] )
            append!( eq, [y[j, k+1] - (dt * (s[j, k] * vx[j, k] + c[j, k] * vy[j, k]) + y[j, k])] )
            
            ## sin cos constraints
            append!( eq, [ s[j,k+1]^2 +  c[j,k+1]^2 - 1.0 ])
            append!( eq, [sa[j,k+1]^2 + ca[j,k+1]^2 - 1.0 ])

            ######## kinematics constraints ########
            if j == 1
                append!( eq, [c[j,k+1] * lx - s[j,k+1] * ly + x[j, k+1] - x[end, k+1] ])
                append!( eq, [s[j,k+1] * lx + c[j,k+1] * ly + y[j, k+1] - 0 ])
            else
                append!( eq, [c[j,k+1] * lx - s[j,k+1] * ly + x[j, k+1] - x[j-1, k+1] ])
                append!( eq, [s[j,k+1] * lx + c[j,k+1] * ly + y[j, k+1] - y[j-1, k+1] ])
            end

            ######## propagation of sdynamics ########
            ## rotation
            Jx1 = (-c[j, k+1] * ly - s[j, k+1] * lx)
            Jy1 = (-s[j, k+1] * ly + c[j, k+1] * lx)
            append!( eq, [(sa[j, k+1] - sa[j, k]) * I - (Jx1 * lam_x[j, k] + Jy1 * lam_y[j, k]) * dt^2] )
            
            ## position
            Fx1 =   c[j, k+1]*lam_x[j, k] + s[j, k+1]*lam_y[j, k]
            Fy1 = - s[j, k+1]*lam_x[j, k] + c[j, k+1]*lam_y[j, k]

            if j < dof
                Fx2 =   c[j, k+1]*lam_x[j+1, k] + s[j+1, k+1]*lam_y[j, k]
                Fy2 = - s[j, k+1]*lam_x[j+1, k] + c[j+1, k+1]*lam_y[j, k]
            else
                Fx2 = 0.
                Fy2 = 0.
            end
            
            append!( eq, [m * vx[j, k+1] - m * ( ca[j, k] * vx[j, k] + sa[j, k] * vy[j, k] ) - (Fx1 + Fx2 + m * s[1, k+1] * g)* dt ] )
            append!( eq, [m * vy[j, k+1] - m * (-sa[j, k] * vx[j, k] + ca[j, k] * vy[j, k] ) - (Fy1 + Fy2 + m * c[1, k+1] * g)* dt ] )

            f = f + ( x[j, k]^2 + (y[j, k] - j * abs(ly))^2 +  (c[j, k] - 1)^2 +  s[j, k]^2) * cartpole_param.run_cost[1]
            f = f + (vx[j, k]^2 + vx[j, k]^2 + vy[j, k]^2   + (ca[j, k] - 1)^2 + sa[j, k]^2)  * cartpole_param.run_cost[1]
            f = f + cartpole_param.run_cost[1] * tau[k]^2
        end

        f = f + ( x[end, k]^2 + vx[end, k]^2) * cartpole_param.run_cost[1]
    end

    # println("diao")
    p_goal = [0; dof]
    
    if cartpole_param.terminal_cons
        # cart states
        append!( eq, [ x[2, 1] - 0.0])
        append!( eq, [vx[2, 1] - 0.0])

        # pole states
        for k in 1:dof
            append!(eq, [ c[k, 1] - cos(t0[k])])
            append!(eq, [ s[k, 1] - sin(t0[k])])
            append!(eq, [vx[k, 1] - 0.0])
            append!(eq, [vy[k, 1] - 0.0])
            append!(eq, [ca[k, 1] - 1.0])
            append!(eq, [sa[k, 1] - 0.0])

            if k == 1
                append!(eq, [cos(0.) * lx - sin(0.) * ly + x[k, 1] - x[end, 1] ])
                append!(eq, [sin(0.) * lx + cos(0.) * ly + y[k, 1] - 0 ])
            else
                append!(eq, [cos(0.) * lx - sin(0.) * ly + x[k, 1] - x[k-1, 1]])
                append!(eq, [sin(0.) * lx + cos(0.) * ly + y[k, 1] - y[k-1, 1]])
            end
        end
    else
        f = f + ( x[end, end]^2 + vx[end, end]^2) * cartpole_param.run_cost[1]
        for j in 1:dof
            f = f + ( x[j, end]^2 + (y[j, end] - j * abs(ly))^2 +  (c[j, end] - 1)^2 +  s[j, end]^2) * cartpole_param.run_cost[1]
            f = f + (vx[j, end]^2 + vx[j, end]^2 + vy[j, end]^2 + (ca[j, end] - 1)^2 + sa[j, end]^2) * cartpole_param.run_cost[1]
        end
    end
    
    # for k = 1:length(var)-Ns
        # append!( ineq, [100 - var[k].^2 ])
        # append!( ineq, [100 - var[k] ])
        # append!( ineq, [100 + var[k]])
    # end

    for k = 1:Ns+1
       append!( ineq, [0 + ca[1, k] ])
    end

    pop = append!([f], ineq, eq)

    order = 2

    eq_len = length(eq)

    time_start = time()
    opt,sol,data,model,sol_approx =cs_tssos_first_export_tsl(pop, var, order, numeq=eq_len, CS = "MF", TS = false, solution=true, QUIET=false, MomentOne=false)
    time_end = time()
    
    elapsed = time_end - time_start
    
    flag = primal_status(model)
    flag = string(flag)
    
    flag2 = termination_status(model)
    flag2 = string(flag2)

    
    moment = []
    for k = 1:length(data.Mmatrix)
        append!(moment, [convert(Matrix{Float64}, data.Mmatrix[k])]) # data.Mmatrix[k])
    end

    return Dict("sol" => sol, "sol_approx" => sol_approx,
                "Ns" => Ns, "dof" => dof, "Ns" => Ns, "moment" => moment,
                "opt" => opt, "param" => cartpole_param, "elapsed" => elapsed,
                "primal" => flag, "termination" => flag2)
    
end

CartPoleTO (generic function with 1 method)

In [6]:
# mpc_param      = Control.param_base(0.125, 8, 1)
mpc_param      = Control.param_base(0.25, 20, 1)
weights        = ones(11, 1)

dof = 2
t0 = Matrix{Float64}(undef, dof, 1)
t0[:] .= pi / 4

for m = [0.5, 0.25]
    cartpole_param = Control.param_cart_multi(mpc_param, m, m, 0.0, -m, weights, 200 * weights, false, 0, t0, dof)
    log = CartPoleTO(cartpole_param)

    if m == 0.5
        matwrite("log_050_eq.mat", log)
    end

    if m == 0.25
        matwrite("log_025_eq.mat", log)
    end
end


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 2]
[9, 18, 18, 9, 16, 9, 1, 2, 5, 1, 19, 47, 29, 19, 4, 3, 9, 3, 12, 2, 2, 21, 2]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.404388 seconds. The maximal size of cliques is 25.
Assembling the SDP...
There are 664108 affine constraints.
SDP assembling time: 9.8136156 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 664108          
  Cones                  : 0               
  Scalar variables       : 63504           
  Matr

In [7]:
for m = [0.4, 0.75]
    cartpole_param = Control.param_cart_multi(mpc_param, m, m, 0.0, -m, weights, 200 * weights, false, 0, t0, dof)
    log = CartPoleTO(cartpole_param)

    if m == 0.4
        matwrite("log_040_0125.mat", log)
    end

    if m == 0.75
        matwrite("log_075_0125.mat", log)
    end
end

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 2]
[9, 18, 18, 9, 16, 9, 1, 2, 5, 1, 19, 47, 29, 19, 4, 3, 9, 3, 12, 2, 2, 21, 2]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.4196878 seconds. The maximal size of cliques is 25.
Assembling the SDP...
There are 664108 affine constraints.
SDP assembling time: 8.9717231 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 664108          
  Cones                  : 0               
  Scalar variables       : 63504           
  Mat

TODO: Find a way to compute in parallel, modify the thread number for usage on vino

In [4]:
string(2)
# "log.mat" * "2"

"2"